# Lab: APIs

##### Instructions: 

Write your own Python script to answer the following questions: 
1. Which of these embassies is closest to the White House in meters? What is the address? 
2. If I wanted to hold a morning meeting there, which cafe would you suggest (best rating and closest)?
3. If I wanted to hold an upscale evening meeting there, which fancy bar would you suggest? 

Hint: 
- You will need to enable the `Google Places API`.
- You may find this page useful to learn about different findinging nearby places https://www.geeksforgeeks.org/python-fetch-nearest-hospital-locations-using-googlemaps-api/

In [76]:
import importlib
import os
import googlemaps
import numpy as np
from googleplaces import GooglePlaces, types, lang 

os.chdir('/home/nath/washu/python_camp/PythonCamp2024/Day05/Lecture')
imported_items = importlib.import_module('start_google')
gmaps = imported_items.client

whitehouse = '1600 Pennsylvania Avenue, Washington, DC'

embassies = [[38.917228,-77.0522365], 
	[38.9076502, -77.0370427], 
	[38.916944, -77.048739] ]

wh = gmaps.geocode('1600 Pennsylvania Avenue, Washington, DC')
wh_loc = wh[0]['geometry']['location']

embassy_names = []
distances = []

for embassy in embassies:
    embassy_loc = {'lat': embassy[0], 'lng': embassy[1]}
    distance = gmaps.distance_matrix(wh_loc, embassy_loc)
    distances.append(distance['rows'][0]['elements'][0]['distance']['value'])
    name = gmaps.reverse_geocode(embassy_loc)[0]['address_components'][0]['long_name'] + \
        " " + \
        gmaps.reverse_geocode(embassy_loc)[0]['address_components'][1]['long_name']
    embassy_names.append(name)

min_idx = np.argmin(np.array(distances))

min_dist_embassy = embassy_names[min_idx]
min_dist_embassy

'1601 Massachusetts Avenue Northwest'

Define functions to fetch the cafes and bars closest to the White House.

In [58]:
def get_closest_cafe():

    """ Gets all cafes within a radius of 5000 units of the White House. """
    
    names, ratings, locations, distances_from_wh = [], [], [], []
    
    query_result = imported_items.google_places.nearby_search( 
            lat_lng = wh_loc, 
            radius = 5000, 
            types =[types.TYPE_CAFE]) 
    
    for place in query_result.places: 
        place.get_details()
        rating = place.rating
        name = place.name
        lat = float(place.geo_location['lat'])
        lng = float(place.geo_location['lng'])
        location = {'lat': lat, 'lng': lng}
        distance = gmaps.distance_matrix(wh_loc,location)
        distance_from_wh = float(distance['rows'][0]['elements'][0]['distance']['value'])
    
        names.append(name)
        ratings.append(rating)
        locations.append(location)
        distances_from_wh.append(distance_from_wh)

    return names, ratings, locations, distances_from_wh  

def get_closest_bar():

    """ Gets all bars within a radius of 5000 units of the White House. """
    
    names, ratings, locations, distances_from_wh = [], [], [], []
    
    query_result = imported_items.google_places.nearby_search( 
            lat_lng = wh_loc, 
            radius = 5000, 
            types =[types.TYPE_BAR]) 
    
    for place in query_result.places: 
        place.get_details()
        rating = place.rating
        name = place.name
        lat = float(place.geo_location['lat'])
        lng = float(place.geo_location['lng'])
        location = {'lat': lat, 'lng': lng}
        distance = gmaps.distance_matrix(wh_loc,location)
        distance_from_wh = float(distance['rows'][0]['elements'][0]['distance']['value'])
    
        names.append(name)
        ratings.append(rating)
        locations.append(location)
        distances_from_wh.append(distance_from_wh)

    return names, ratings, locations, distances_from_wh  

Choose top 20 closest cafes. Then out of those top 20 closest cafes, select the cafe with the best rating.

In [61]:
names, ratings, locations, distances_from_wh = get_closest_cafe()

import pandas as pd

df = pd.DataFrame({'name': names,
                   'rating': ratings,
                   'location': locations,
                   'distance_from_wh': distances_from_wh})

df = df.sort_values(by='distance_from_wh', ascending=True)
df = df.head(20)
max_row = df.loc[df['rating'].idxmax()]
max_row

name                                          Le Diplomate
rating                                                 4.6
location            {'lat': 38.911365, 'lng': -77.0316635}
distance_from_wh                                    3164.0
Name: 2, dtype: object

Get top 20 closest bars. Of these 20 bars, select the bar with the highest rating.

In [62]:
names, ratings, locations, distances_from_wh = get_closest_bar()

import pandas as pd

df = pd.DataFrame({'name': names,
                   'rating': ratings,
                   'location': locations,
                   'distance_from_wh': distances_from_wh})

df = df.sort_values(by='distance_from_wh', ascending=True)
df = df.head(20)
max_row = df.loc[df['rating'].idxmax()]
max_row

name                                 The Jefferson, Washington, DC
rating                                                         4.7
location            {'lat': 38.9059541, 'lng': -77.03683029999999}
distance_from_wh                                            2942.0
Name: 0, dtype: object